In [1]:
import sys
import json
import pandas as pd
from pandas.io.json import json_normalize
sys.path.append("../") # Enables importing from parent directory
import fpl_draft_league as fpl
import transactions as trxns
import matplotlib.pyplot as plt
import datetime

In [11]:
league_entry_df, matches_df, standings_df = fpl.get_dataframes('../data/fpl_data.json')

In [12]:
# Filter to played matches
matches_df = matches_df[matches_df['finished'] == True]

# Join to get team names and player names of entry 1 (home team)
matches_df = pd.merge(matches_df,
                      league_entry_df[['id', 'player_first_name']],
                      how='left',
                      left_on='league_entry_1',
                      right_on='id')

# Join to get team names and player names of entry 2 (away team)
matches_df = pd.merge(matches_df,
                      league_entry_df[['id', 'player_first_name']],
                      how='left',
                      left_on='league_entry_2',
                      right_on='id')

# Drop unused columns, rename for clearer columns
matches_df = (matches_df
             .drop(['finished', 'started', 'id_x', 'id_y', 'league_entry_1', 'league_entry_2', 'winning_league_entry', 'winning_method'], axis=1)
            .rename(columns={'event':'match',
                       'player_first_name_x': 'home_player',
                       'league_entry_1_points': 'home_score',
                       'player_first_name_y': 'away_player',
                       'league_entry_2_points': 'away_score',
                      })
            )

matches_df

,match,home_score,away_score,home_player,away_player
0,1,46,43,Huw,Benji
1,1,36,30,Dave,Thomas
2,1,40,30,ben,Liam
3,1,69,51,Cory,James
4,1,23,68,Rebecca,John
5,2,25,24,Benji,Rebecca
6,2,39,31,John,Cory
7,2,29,36,James,ben
8,2,21,37,Liam,Dave
9,2,39,31,Thomas,Huw


In [13]:
def calc_points(df):
    df['home_margin'] = df['home_score'] - df['away_score']
    df['away_margin'] = df['away_score'] - df['home_score']

    return df

In [15]:
matches_df.apply(calc_points, axis=1)

,match,home_score,away_score,home_player,away_player,home_margin,away_margin
0,1,46,43,Huw,Benji,3,-3
1,1,36,30,Dave,Thomas,6,-6
2,1,40,30,ben,Liam,10,-10
3,1,69,51,Cory,James,18,-18
4,1,23,68,Rebecca,John,-45,45
5,2,25,24,Benji,Rebecca,1,-1
6,2,39,31,John,Cory,8,-8
7,2,29,36,James,ben,-7,7
8,2,21,37,Liam,Dave,-16,16
9,2,39,31,Thomas,Huw,8,-8
